# 🍽️ Daily Meal Plan Generator

Generate a healthy, well-balanced daily meal plan using **OpenAI GPT-4o** for recipes and **DALL·E 3** for photorealistic food images.

## Features
- AI-generated Breakfast, Lunch & Dinner recipes
- Precise nutritional info & calorie tracking
- DALL·E 3 photorealistic food photography
- AI video script generation for each recipe
- Interactive Gradio UI

## Setup
1. Go to **Secrets** (🔑 icon in left sidebar) in Google Colab
2. Add a new secret named `OPENAI_API_KEY` with your OpenAI API key
3. Toggle **Notebook access** ON for the secret
4. Run all cells below

## 1. Install Dependencies

In [ ]:
!pip install -q openai gradio

## 2. Configure OpenAI API Key

In [ ]:
import os
import getpass
from openai import OpenAI

# Try Colab Secrets first, fall back to manual input
api_key = None
try:
    from google.colab import userdata
    api_key = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab Secrets")
except Exception:
    pass

if not api_key:
    api_key = getpass.getpass("Enter your OpenAI API key: ")
    print("✅ API key set manually")

client = OpenAI(api_key=api_key)

# Quick validation
try:
    client.models.list()
    print("✅ OpenAI client connected successfully!")
except Exception as e:
    print(f"❌ Connection failed: {e}")

## 3. Prompt Template & Output Format

In [ ]:
# ---------- Output format schema ----------
OUTPUT_FORMAT = """
For EACH meal (Breakfast, Lunch, Dinner), provide the following sections in this exact order:

MEAL TYPE: [Breakfast / Lunch / Dinner]
TITLE: [Descriptive recipe title]
SERVINGS: [Number]
PREP TIME: [minutes]
COOK TIME: [minutes]
TOTAL TIME: [minutes]
CALORIES: [number] kcal

INGREDIENTS:
- [quantity] [ingredient]
- ...

INSTRUCTIONS:
1. [Step]
2. [Step]
...

DALL-E IMAGE PROMPT:
[Detailed photorealistic food photography prompt]

VIDEO SCRIPT:
[Scene-by-scene short-form cooking video script]

--------------------------------------------------

After the last recipe, on the VERY LAST LINE, print ONLY the three recipe titles separated by commas.
"""

# ---------- Main prompt template ----------
PROMPT_TEMPLATE = """You are a certified nutritionist, professional recipe developer, and food content creator.

Your task is to create a healthy, well-balanced daily meal plan that includes:
- Breakfast
- Lunch
- Dinner

Use the following ingredients as input:
{ingredients}

The final output MUST strictly follow the format: {output_format}

====================
INSTRUCTIONS
====================

1. Ingredient Usage:
   {ingredient_rule}

2. Ingredient Precision:
   - Specify the exact quantity of every ingredient used (grams, milliliters, pieces, etc.).
   - Avoid vague measurements (e.g., "some", "to taste").

3. Caloric Constraint:
   - Ensure the TOTAL daily calorie intake across all meals is strictly below {kcal} kcal.
   - Clearly state calories per meal.

4. Recipe Instructions:
   - For each meal, provide step-by-step cooking instructions.
   - Use numbered steps or bullet points.
   - Keep instructions clear, concise, and easy to follow.

5. Nutritional Breakdown:
   - For each meal, specify:
     - Total calories
     - Number of servings

6. Recipe Titles (Image Generation Ready):
   - Provide a concise, descriptive title for each meal.
   - The title must clearly reflect the main ingredients and flavor profile.
   - Each title must be suitable as a DALL-E image generation prompt using a photorealistic food photography style.

7. Meal Image Generation:
   - For each meal, generate a separate DALL-E image prompt.
   - The image prompt must describe:
     - The finished dish
     - Plating style
     - Lighting (e.g., natural light, studio food photography)
     - Camera angle (e.g., top-down, 45-degree angle)
     - High realism and appetizing presentation
   - Do NOT generate the image itself—only the prompt text.

8. Recipe Video Generation:
   - For each meal, generate a short-form cooking video script (30-60 seconds).
   - The video script must include:
     - Scene-by-scene breakdown
     - Visual actions (chopping, cooking, plating)
     - On-screen text suggestions
     - Optional voiceover narration
   - The script should be suitable for AI video tools (e.g., Sora, Runway, Pika, or similar).

9. Time Estimates:
   - For each recipe, clearly specify:
     - Preparation time
     - Cooking time
     - Total time

10. Additional Constraints:
   {extra_rule}

11. Formatting Rules:
   - Separate each recipe using EXACTLY 50 dash characters:
     --------------------------------------------------
   - Do not include extra commentary, explanations, or disclaimers outside the requested structure.

====================
FINAL VALIDATION
====================

Before producing the final answer, internally verify that ALL rules (1-11) are fully satisfied.

====================
FINAL OUTPUT REQUIREMENT
====================

The LAST LINE of your response must contain ONLY the recipe titles, separated by commas, and nothing else.

Example:
Broccoli and Egg Scramble, Grilled Chicken and Vegetables, Baked Fish with Cabbage Slaw"""


def build_prompt(ingredients: str, kcal: int, exact_ingredients: bool, extra: str) -> str:
    """Render the prompt template with user inputs."""
    ingredient_rule = (
        "Use ONLY the provided ingredients. You may also use salt, pepper, herbs, and spices."
        if exact_ingredients
        else "Use the provided ingredients as a base. You may add additional ingredients if necessary to improve flavor, nutritional balance, or recipe completeness."
    )
    extra_rule = (
        f"If possible, the meals should be: {extra}" if extra.strip() else "No additional constraints."
    )
    return PROMPT_TEMPLATE.format(
        ingredients=ingredients,
        output_format=OUTPUT_FORMAT,
        kcal=kcal,
        ingredient_rule=ingredient_rule,
        extra_rule=extra_rule,
    )

print("✅ Prompt template loaded")

## 4. Core Generation & Parsing Functions

In [ ]:
import re
import json
import requests
from io import BytesIO
import base64


# ──────────────────────────────────────────────
# Text generation
# ──────────────────────────────────────────────
def generate_meal_plan(ingredients: str, kcal: int, exact_ingredients: bool, extra: str) -> str:
    """Call GPT-4o to generate a full daily meal plan."""
    prompt = build_prompt(ingredients, kcal, exact_ingredients, extra)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a certified nutritionist, professional recipe developer, "
                    "and food content creator. Follow ALL formatting rules exactly."
                ),
            },
            {"role": "user", "content": prompt},
        ],
        temperature=0.7,
        max_tokens=4096,
    )
    return response.choices[0].message.content


# ──────────────────────────────────────────────
# Parsing
# ──────────────────────────────────────────────
def _extract_field(block: str, label: str) -> str:
    """Extract a single-line field value after 'LABEL: '."""
    match = re.search(rf"{label}:\s*(.+)", block, re.IGNORECASE)
    return match.group(1).strip() if match else ""


def _extract_section(block: str, header: str) -> str:
    """Extract a multi-line section that starts with HEADER: and ends before the next known header."""
    headers = [
        "MEAL TYPE", "TITLE", "SERVINGS", "PREP TIME", "COOK TIME",
        "TOTAL TIME", "CALORIES", "INGREDIENTS", "INSTRUCTIONS",
        "DALL-E IMAGE PROMPT", "VIDEO SCRIPT",
    ]
    # Build alternation of all headers EXCEPT the current one
    others = "|".join(re.escape(h) for h in headers if h != header)
    pattern = rf"{re.escape(header)}:\s*\n(.*?)(?=\n(?:{others}):|\Z)"
    match = re.search(pattern, block, re.DOTALL | re.IGNORECASE)
    return match.group(1).strip() if match else ""


def parse_meal_plan(raw_text: str) -> dict:
    """Parse the GPT response into structured meal data."""
    separator = "-" * 50
    # Split by the 50-dash separator
    parts = re.split(r"-{40,}", raw_text)

    meals = []
    for part in parts:
        part = part.strip()
        if not part or "TITLE:" not in part.upper():
            # Could be the final titles line or empty
            continue
        meal = {
            "meal_type": _extract_field(part, "MEAL TYPE"),
            "title": _extract_field(part, "TITLE"),
            "servings": _extract_field(part, "SERVINGS"),
            "prep_time": _extract_field(part, "PREP TIME"),
            "cook_time": _extract_field(part, "COOK TIME"),
            "total_time": _extract_field(part, "TOTAL TIME"),
            "calories": _extract_field(part, "CALORIES"),
            "ingredients": _extract_section(part, "INGREDIENTS"),
            "instructions": _extract_section(part, "INSTRUCTIONS"),
            "dalle_prompt": _extract_section(part, "DALL-E IMAGE PROMPT"),
            "video_script": _extract_section(part, "VIDEO SCRIPT"),
            "image_url": None,
            "audio_b64": None,
        }
        meals.append(meal)

    # Extract final titles line
    lines = [l.strip() for l in raw_text.strip().splitlines() if l.strip()]
    titles_line = lines[-1] if lines else ""

    return {"meals": meals[:3], "titles": titles_line}


# ──────────────────────────────────────────────
# Image generation
# ──────────────────────────────────────────────
def generate_meal_image(dalle_prompt: str) -> str:
    """Generate a single DALL-E 3 image and return the URL."""
    if not dalle_prompt:
        return None
    response = client.images.generate(
        model="dall-e-3",
        prompt=dalle_prompt,
        size="1024x1024",
        quality="standard",
        n=1,
    )
    return response.data[0].url


def generate_all_images(parsed: dict) -> dict:
    """Generate DALL-E images for all meals."""
    for meal in parsed["meals"]:
        try:
            meal["image_url"] = generate_meal_image(meal["dalle_prompt"])
        except Exception as e:
            print(f"⚠️ Image generation failed for '{meal['title']}': {e}")
            meal["image_url"] = None
    return parsed


# ──────────────────────────────────────────────
# Audio generation (TTS)
# ──────────────────────────────────────────────
def _build_audio_script(meal: dict) -> str:
    """Build a natural narration script from the meal data."""
    parts = []
    parts.append(f"Here is your {meal['meal_type']} recipe: {meal['title']}.")
    parts.append(f"This recipe serves {meal['servings']} and takes {meal['total_time']} in total.")
    parts.append(f"It has {meal['calories']} per serving.")

    # Ingredients
    parts.append("Here are the ingredients you will need:")
    for line in meal["ingredients"].strip().splitlines():
        ingredient = line.strip().lstrip("- ").strip()
        if ingredient:
            parts.append(f"  {ingredient}.")

    # Instructions
    parts.append("Now, let's go through the cooking steps:")
    for line in meal["instructions"].strip().splitlines():
        step = re.sub(r"^\d+\.\s*", "", line.strip()).strip()
        if step:
            parts.append(step)

    parts.append(f"And that's it! Enjoy your {meal['title']}.")
    return "\n".join(parts)


def generate_meal_audio(meal: dict) -> str:
    """Generate TTS audio for a meal recipe and return base64-encoded mp3."""
    script = _build_audio_script(meal)
    try:
        response = client.audio.speech.create(
            model="tts-1",
            voice="nova",
            input=script,
        )
        audio_bytes = response.content
        return base64.b64encode(audio_bytes).decode("utf-8")
    except Exception as e:
        print(f"⚠️ Audio generation failed for '{meal['title']}': {e}")
        return None


print("✅ Generation, parsing & audio functions loaded")

## 5. HTML Rendering

In [ ]:
import html as html_lib

MEAL_EMOJIS = {"Breakfast": "🌅", "Lunch": "☀️", "Dinner": "🌙"}


def _nl2br(text: str) -> str:
    """Convert newlines to <br> tags, escaping HTML."""
    return html_lib.escape(text).replace("\n", "<br>")


def _format_ingredients(text: str) -> str:
    """Convert ingredient lines to an HTML list."""
    lines = [l.strip().lstrip("- ") for l in text.strip().splitlines() if l.strip()]
    items = "".join(f"<li>{html_lib.escape(l)}</li>" for l in lines)
    return f"<ul>{items}</ul>"


def _format_instructions(text: str) -> str:
    """Convert numbered instruction lines to an HTML ordered list."""
    lines = [re.sub(r"^\d+\.\s*", "", l.strip()) for l in text.strip().splitlines() if l.strip()]
    items = "".join(f"<li>{html_lib.escape(l)}</li>" for l in lines)
    return f"<ol>{items}</ol>"


def _render_audio_player(meal: dict) -> str:
    """Render an HTML audio player if audio is available."""
    if not meal.get("audio_b64"):
        return ""
    return f"""
    <div style="margin-top:16px;background:#f0f4ff;padding:14px;border-radius:10px;
                border:1px solid #d0d8f0;">
        <div style="display:flex;align-items:center;gap:8px;margin-bottom:8px;">
            <span style="font-size:18px;">🔊</span>
            <span style="font-weight:600;color:#4a6cf7;font-size:14px;">Listen to Recipe</span>
        </div>
        <audio controls style="width:100%;border-radius:8px;">
            <source src="data:audio/mp3;base64,{meal['audio_b64']}" type="audio/mpeg">
            Your browser does not support the audio element.
        </audio>
    </div>"""


def render_meal_card(meal: dict, index: int) -> str:
    """Render a single meal as a styled HTML card."""
    emoji = MEAL_EMOJIS.get(meal["meal_type"], "🍽️")
    image_html = (
        f'<img src="{meal["image_url"]}" alt="{html_lib.escape(meal["title"])}" '
        f'style="width:100%;border-radius:12px 12px 0 0;object-fit:cover;max-height:350px;">'
        if meal.get("image_url")
        else '<div style="height:200px;background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);'
             'border-radius:12px 12px 0 0;display:flex;align-items:center;justify-content:center;'
             f'font-size:48px;">{emoji}</div>'
    )

    audio_html = _render_audio_player(meal)

    return f"""
    <div style="background:#fff;border-radius:12px;box-shadow:0 4px 20px rgba(0,0,0,0.08);
                overflow:hidden;margin-bottom:24px;border:1px solid #e8e8e8;">
        {image_html}
        <div style="padding:20px;">
            <div style="display:flex;align-items:center;gap:8px;margin-bottom:8px;">
                <span style="background:#f0f4ff;color:#4a6cf7;padding:4px 12px;border-radius:20px;
                             font-size:13px;font-weight:600;">{emoji} {html_lib.escape(meal['meal_type'])}</span>
                <span style="background:#fff3e0;color:#e65100;padding:4px 12px;border-radius:20px;
                             font-size:13px;font-weight:600;">🔥 {html_lib.escape(meal['calories'])}</span>
            </div>
            <h2 style="margin:8px 0 12px;color:#1a1a2e;font-size:22px;">{html_lib.escape(meal['title'])}</h2>

            <div style="display:flex;gap:12px;flex-wrap:wrap;margin-bottom:16px;">
                <span style="background:#f5f5f5;padding:4px 10px;border-radius:8px;font-size:12px;">⏱ Prep: {html_lib.escape(meal['prep_time'])}</span>
                <span style="background:#f5f5f5;padding:4px 10px;border-radius:8px;font-size:12px;">🍳 Cook: {html_lib.escape(meal['cook_time'])}</span>
                <span style="background:#f5f5f5;padding:4px 10px;border-radius:8px;font-size:12px;">⏰ Total: {html_lib.escape(meal['total_time'])}</span>
                <span style="background:#f5f5f5;padding:4px 10px;border-radius:8px;font-size:12px;">🍽 Servings: {html_lib.escape(meal['servings'])}</span>
            </div>

            {audio_html}

            <h3 style="color:#333;border-bottom:2px solid #4a6cf7;padding-bottom:4px;">📝 Ingredients</h3>
            {_format_ingredients(meal['ingredients'])}

            <h3 style="color:#333;border-bottom:2px solid #4a6cf7;padding-bottom:4px;">👩‍🍳 Instructions</h3>
            {_format_instructions(meal['instructions'])}

            <details style="margin-top:16px;background:#fafafa;padding:12px;border-radius:8px;">
                <summary style="cursor:pointer;font-weight:600;color:#555;">🎥 Video Script</summary>
                <div style="margin-top:8px;font-size:14px;line-height:1.6;color:#555;">
                    {_nl2br(meal['video_script'])}
                </div>
            </details>

            <details style="margin-top:8px;background:#fafafa;padding:12px;border-radius:8px;">
                <summary style="cursor:pointer;font-weight:600;color:#555;">🖼️ DALL·E Prompt</summary>
                <div style="margin-top:8px;font-size:13px;line-height:1.5;color:#777;font-style:italic;">
                    {_nl2br(meal['dalle_prompt'])}
                </div>
            </details>
        </div>
    </div>"""


def render_full_plan(parsed: dict, kcal_target: int) -> str:
    """Render the complete meal plan as styled HTML."""
    # Calculate total calories
    total_cal = 0
    for meal in parsed["meals"]:
        cal_match = re.search(r"(\d+)", meal.get("calories", "0"))
        if cal_match:
            total_cal += int(cal_match.group(1))

    cal_color = "#4caf50" if total_cal < kcal_target else "#f44336"

    cards = "".join(render_meal_card(m, i) for i, m in enumerate(parsed["meals"]))

    return f"""
    <div style="font-family:'Segoe UI',Roboto,sans-serif;max-width:800px;margin:0 auto;padding:20px;">
        <div style="text-align:center;margin-bottom:32px;">
            <h1 style="color:#1a1a2e;margin-bottom:8px;">🍽️ Your Daily Meal Plan</h1>
            <div style="display:inline-flex;gap:16px;align-items:center;">
                <span style="font-size:18px;color:{cal_color};font-weight:700;">
                    Total: {total_cal} / {kcal_target} kcal
                </span>
                <span style="font-size:14px;color:#888;">{'✅ Under target' if total_cal < kcal_target else '⚠️ Over target'}</span>
            </div>
        </div>
        {cards}
        <div style="text-align:center;margin-top:20px;padding:16px;background:#f0f4ff;border-radius:10px;">
            <strong>Recipe Titles:</strong> {html_lib.escape(parsed.get('titles', ''))}
        </div>
    </div>"""


print("✅ HTML rendering functions loaded")

## 6. Gradio Interface

In [ ]:
import gradio as gr


def orchestrate(ingredients: str, kcal: int, exact_ingredients: bool, extra: str,
                generate_images: bool, generate_audio: bool, progress=gr.Progress()):
    """Full pipeline: prompt → GPT-4o → parse → DALL-E 3 → TTS audio → HTML."""
    if not ingredients.strip():
        return "<p style='color:red;text-align:center;'>⚠️ Please enter at least one ingredient.</p>"

    # Step 1: Generate meal plan text
    progress(0.1, desc="🧠 Generating meal plan with GPT-4o...")
    try:
        raw_text = generate_meal_plan(ingredients, kcal, exact_ingredients, extra)
    except Exception as e:
        return f"<p style='color:red;'>❌ GPT-4o error: {e}</p>"

    # Step 2: Parse response
    progress(0.25, desc="📋 Parsing recipes...")
    parsed = parse_meal_plan(raw_text)

    if not parsed["meals"]:
        return (
            "<p style='color:red;'>❌ Failed to parse the meal plan. Raw response:</p>"
            f"<pre style='white-space:pre-wrap;'>{html_lib.escape(raw_text)}</pre>"
        )

    # Step 3: Generate images (optional)
    if generate_images:
        for i, meal in enumerate(parsed["meals"]):
            progress(0.3 + i * 0.12, desc=f"🎨 Generating image {i+1}/3: {meal['title']}...")
            try:
                meal["image_url"] = generate_meal_image(meal["dalle_prompt"])
            except Exception as e:
                print(f"⚠️ Image failed for '{meal['title']}': {e}")
                meal["image_url"] = None

    # Step 4: Generate audio narration (optional)
    if generate_audio:
        for i, meal in enumerate(parsed["meals"]):
            progress(0.7 + i * 0.08, desc=f"🔊 Generating audio {i+1}/3: {meal['title']}...")
            meal["audio_b64"] = generate_meal_audio(meal)

    # Step 5: Render HTML
    progress(0.95, desc="✨ Rendering...")
    result_html = render_full_plan(parsed, kcal)

    progress(1.0, desc="✅ Done!")
    return result_html


# ──────────────────────────────────────────────
# Build Gradio UI
# ──────────────────────────────────────────────
with gr.Blocks(
    title="Daily Meal Plan Generator",
    theme=gr.themes.Soft(primary_hue="blue", secondary_hue="orange"),
    css=".gradio-container {max-width: 1000px !important;}",
) as demo:
    gr.Markdown(
        """# 🍽️ Daily Meal Plan Generator
        **Powered by GPT-4o, DALL·E 3 & TTS** — Enter your ingredients and get a complete,
        calorie-controlled meal plan with photorealistic images, audio narration, and video scripts."""
    )

    with gr.Row():
        with gr.Column(scale=1):
            ingredients_input = gr.Textbox(
                label="🥗 Ingredients",
                placeholder="e.g., chicken breast, broccoli, rice, olive oil, eggs, spinach, tomatoes",
                lines=4,
                info="Enter the ingredients you have available, separated by commas.",
            )
            kcal_slider = gr.Slider(
                minimum=1200, maximum=3500, value=2000, step=50,
                label="🔥 Max Daily Calories (kcal)",
            )
            with gr.Row():
                exact_checkbox = gr.Checkbox(
                    label="🔒 Exact ingredients only",
                    value=False,
                    info="If checked, only your listed ingredients will be used (plus basic seasonings).",
                )
                images_checkbox = gr.Checkbox(
                    label="🖼️ Generate DALL·E images",
                    value=True,
                    info="Generates 3 food photos (~$0.12 total). Uncheck to save cost.",
                )
            audio_checkbox = gr.Checkbox(
                label="🔊 Generate audio narration",
                value=True,
                info="Generates TTS audio for each recipe (~$0.03 total). Listen to ingredients & cooking steps.",
            )
            extra_input = gr.Textbox(
                label="📌 Additional Constraints (optional)",
                placeholder="e.g., high-protein, vegetarian, low-sodium, Mediterranean-style",
                lines=2,
            )
            generate_btn = gr.Button("🚀 Generate Meal Plan", variant="primary", size="lg")

    output_html = gr.HTML(label="Meal Plan")

    generate_btn.click(
        fn=orchestrate,
        inputs=[ingredients_input, kcal_slider, exact_checkbox, extra_input, images_checkbox, audio_checkbox],
        outputs=output_html,
    )

    gr.Markdown(
        """---
        <center style='color:#888;font-size:12px;'>
        Built with ❤️ using OpenAI GPT-4o, DALL·E 3, TTS, and Gradio<br>
        Each generation uses ~1 GPT-4o call + 3 DALL·E 3 calls + 3 TTS calls
        </center>"""
    )

# Launch
demo.launch(debug=True)